## Predict stock price for S&P 500 using historical performance
In this project, you'll work with data from the S&P500 Index.
Indexes aggregate the prices of multiple stocks together, and allow you to see how the market as a whole performs.


The S&P500 Index aggregates the stock prices of 500 large companies. When an index fund goes up or down, you can say that the primary market or sector it represents is doing the same. For example, if the Dow Jones Industrial Average price goes down one day, you can say that American stocks overall went down (ie, most American stocks went down in price).

You'll be using historical data on the price of the S&P500 Index to make predictions about future prices. Predicting whether an index goes up or down helps forecast how the stock market as a whole performs. Since stocks tend to correlate with how well the economy as a whole is performs, it can also help with economic forecasts.

The purpose of this notebook is to setup a workflow for evaluating different models with varying features to obtain the best prediction of future performance. The full analysis will be carreid through for a linear regression model and then other models will be considered for comparison. In the inner loop, features will be progressively added and the model performance evaluated to determine which model/features provide the best prediction.

Two predictions will be made: a one-step ahead prediction, and a multi-step ahead prediction based on a 95%-5% data split.

### A few notes on modeling time series data
Obtaining reliable time series predictions is possible if the dependence between values in the past is preserved in the future. This means that the stochastic properties underlying the data do not change with shifts in time. This implies that the first two moments are finite, thus the mean and variance are constant, and the covariance only depends on the time shift and not the start time.

Two popular approaches that will be considered here are: <br><br>
1) nearest neighbour one-step-ahead (use previous n points to predict the value at n+1 - similar to a traditional regression scenario) <br>
2) multi-step ahead: multiple one-step-ahead regressors are used to predict the value at h steps ahead. This can be iterative, i.e.: the prediction at t+1 is used as the input to predicting t+2, or only previous values are used, or direct, i.e.: the value at t+h is predicted directly from previous values.

## Load and format data

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

# read in stock prices
stocks = pd.read_csv('sphist.csv')

# convert Date columns to date-time format
stocks['Date'] = pd.to_datetime(stocks['Date'])

# sort by ascending date (default of sort_values)
stocks.sort_values(by =['Date'], axis = 0, inplace = True)
# re-index with default values to keep original index
stocks.reset_index(inplace = True)

# show data types and check quickly for missing values
#stocks.info()
stocks.head(5)

,index,Date,Open,High,Low,Close,Volume,Adj Close
0,16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
1,16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
2,16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
3,16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
4,16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


Datasets taken from the stock market need to be handled differently than datasets from other sectors when it's time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential and each observation comes a day after the previous observation. **Thus, the observations are not all independent and you can't treat them as such.**

This means you have to be extra careful not to inject "future" knowledge into past rows when you train and predict. Injecting future knowledge makes our model look good when we train and test it, but fails to generalize well.

Here we will look at metrics for price and volume over 3 different time periods (5 days, 30 days, 365 days):

1) The average price and volume over the past 5, 30 and 365 days, <br>
2) The standard deviation of price and volume over the past 5, 30 and 365 days, <br>
3) The ratio between the average price/volume for the past 5 days, and the average price/volume for the past 365 days, <br>
4) The ratio between the standard deviation of price/volume for the past 5 days, and the standard deviation of price/volume for the past 365 days, <br>
5) The ratio between the average price/volume for the past 30 days, and the average price/volume for the past 365 days, <br>
6) The ratio between the standard deviation of price/volume for the past 30 days, and the standard deviation of price/volume for the past 365 days, <br>

Assume that ''days'' means trading days and the price is denoted by ''Close''.

In [2]:
# ----- compute a stock indicators -----        
# here we use the rolling functionality in pandas to performed windowed calculations rolling selects a sliding window
# of N points. We calculate the mean of this window and then use the shift function to slide the average one day
# so as to not include the current day in the calculations (data leakage). 

# rolling average of price and volume
stocks["avg_price_5"] = stocks["Close"].rolling(5).mean().shift(1)
stocks["avg_price_30"] = stocks["Close"].rolling(30).mean().shift(1)
stocks["avg_price_365"] = stocks["Close"].rolling(365).mean().shift(1)
stocks["avg_vol_5"] = stocks["Volume"].rolling(5).mean().shift(1)
stocks["avg_vol_30"] = stocks["Volume"].rolling(30).mean().shift(1)
stocks["avg_vol_365"] = stocks["Volume"].rolling(365).mean().shift(1)

# same procedure for std calcultions
stocks["std_price_5"] = stocks["Close"].rolling(5).std().shift(1)
stocks["std_price_30"] = stocks["Close"].rolling(30).std().shift(1)
stocks["std_price_365"] = stocks["Close"].rolling(365).std().shift(1)
stocks["std_vol_5"] = stocks["Volume"].rolling(5).std().shift(1)
stocks["std_vol_30"] = stocks["Volume"].rolling(30).std().shift(1)
stocks["std_vol_365"] = stocks["Volume"].rolling(365).std().shift(1)

# compute ratios 
stocks["ratio_avg_price_5"] = stocks["avg_price_5"] / stocks["avg_price_365"]
stocks["ratio_avg_price_30"] = stocks["avg_price_30"] / stocks["avg_price_365"]
stocks["ratio_avg_vol_5"] = stocks["avg_vol_5"] / stocks["avg_vol_365"]
stocks["ratio_avg_vol_30"] = stocks["avg_price_30"] / stocks["avg_vol_365"]
stocks["ratio_std_price_5"] = stocks["std_price_5"] / stocks["std_price_365"]
stocks["ratio_std_price_30"] = stocks["std_price_30"] / stocks["std_price_365"]
stocks["ratio_std_vol_5"] = stocks["std_vol_5"] / stocks["std_vol_365"]
stocks["ratio_std_vol_30"] = stocks["std_vol_30"] / stocks["std_vol_365"]

# remove rows where data is missing (within 1 year of start date)
stocks = stocks[stocks['Date'] > datetime(year=1951, month=1, day=2)]

# remove any data with nans
stocks.dropna(axis=0, inplace = True)

# drop open, high, low, adj close and old index
cols_to_drop = ['Open', 'High', 'Low', 'Adj Close', 'index']
stocks.drop(columns = cols_to_drop, inplace = True)

# print first 10 rows of dataframe
stocks.head(10)


,Date,Close,Volume,avg_price_5,avg_price_30,avg_price_365,avg_vol_5,avg_vol_30,avg_vol_365,std_price_5,...,std_vol_30,std_vol_365,ratio_avg_price_5,ratio_avg_price_30,ratio_avg_vol_5,ratio_avg_vol_30,ratio_std_price_5,ratio_std_price_30,ratio_std_vol_5,ratio_std_vol_30
365,1951-06-19,22.020000,1100000.0,21.800,21.703333,19.447726,1196000.0,1.707667e+06,1.989479e+06,0.256223,...,1.566790e+06,772310.649463,1.120954,1.115983,0.601162,0.000011,0.143121,0.264541,0.184164,2.028705
366,1951-06-20,21.910000,1120000.0,21.900,21.683000,19.462411,1176000.0,1.691667e+06,1.989041e+06,0.213659,...,1.570585e+06,772771.102512,1.125246,1.114096,0.591240,0.000011,0.119409,0.248503,0.192069,2.032407
367,1951-06-21,21.780001,1100000.0,21.972,21.659667,19.476274,1188000.0,1.675667e+06,1.986932e+06,0.092574,...,1.573993e+06,774092.100761,1.128142,1.112105,0.597907,0.000011,0.051758,0.230040,0.179328,2.033340
368,1951-06-22,21.549999,1340000.0,21.960,21.631000,19.489562,1148000.0,1.647000e+06,1.982959e+06,0.115108,...,1.576465e+06,774914.749625,1.126757,1.109876,0.578933,0.000011,0.064390,0.206143,0.163589,2.034372
369,1951-06-25,21.290001,2440000.0,21.862,21.599000,19.502082,1142000.0,1.636333e+06,1.981123e+06,0.204132,...,1.577456e+06,775643.723180,1.121008,1.107523,0.576441,0.000011,0.114293,0.184279,0.146544,2.033738
370,1951-06-26,21.299999,1260000.0,21.710,21.564333,19.513617,1420000.0,1.663000e+06,1.980904e+06,0.292831,...,1.584268e+06,775502.321512,1.112556,1.105092,0.716844,0.000011,0.164148,0.169978,0.746790,2.042892
371,1951-06-27,21.370001,1360000.0,21.566,21.535000,19.525315,1452000.0,1.663333e+06,1.978438e+06,0.278980,...,1.584179e+06,776361.635923,1.104515,1.102927,0.733912,0.000011,0.156589,0.159151,0.722830,2.040517
372,1951-06-28,21.100000,1940000.0,21.458,21.522000,19.537041,1500000.0,1.641333e+06,1.974959e+06,0.208015,...,1.583638e+06,776279.384225,1.098324,1.101600,0.759509,0.000011,0.116887,0.158353,0.689636,2.040036
373,1951-06-29,20.959999,1730000.0,21.322,21.502333,19.548932,1668000.0,1.650667e+06,1.972137e+06,0.162080,...,1.584576e+06,774522.421356,1.090699,1.099924,0.845783,0.000011,0.091286,0.163415,0.657989,2.045875
374,1951-07-02,21.100000,1350000.0,21.204,21.470667,19.560685,1746000.0,1.662667e+06,1.967753e+06,0.169205,...,1.583741e+06,771337.424575,1.084011,1.097644,0.887306,0.000011,0.095564,0.167120,0.616570,2.053240


## Split data for training and testing

In [4]:
# split data based on first day of 2013
train = stocks[stocks['Date'] < datetime(year=2013, month=1, day=1)]
test = stocks[stocks['Date'] >= datetime(year=2013, month=1, day=1)]

print()
print("train: {} values ==> {}%".format(len(train), round((len(train) / len(stocks)) * 100, 2)))
print()
print("test: {} values =====>  {}%".format(len(test), round((len(test) / len(stocks)) * 100, 2)))


train: 15486 values ==> 95.45%

test: 739 values =====>  4.55%
